In [12]:
import pandas as pd
import requests
import json
pd.options.mode.copy_on_write = True
url = "https://api.cartola.globo.com/atletas/mercado"
response = requests.get(url)
json_data = response.json()
data_dict = json.loads(response.text)
atletas_df = pd.json_normalize(data_dict["atletas"],max_level=2)
atletas_df=atletas_df.add_prefix('atleta_')
print(atletas_df.columns.tolist())
atletas_df = atletas_df.rename(columns={'atleta_atleta_id':'atleta_id',
                                        'atleta_clube_id': 'clube_id',
                                        'atleta_posicao_id':'posicao_id',
                                        'atleta_status_id':'status_id'})

# Create the clubes DataFrame
clubes_df = pd.DataFrame.from_dict(data_dict["clubes"], orient='index')
clubes_df=clubes_df.add_prefix('clube_')
print(clubes_df.columns.tolist())

# Create the posicoes DataFrame
posicoes_df = pd.DataFrame.from_dict(data_dict["posicoes"], orient='index')
posicoes_df=posicoes_df.add_prefix('posicao_')
print(posicoes_df.columns.tolist())

#Create the status dataframe
status_df=pd.DataFrame.from_dict(data_dict["status"], orient='index')
status_df=status_df.add_prefix('status_')
print(status_df.columns.tolist())

colunas = ['atleta_apelido','clube_abreviacao','posicao_abreviacao','status_nome', 'atleta_preco_num']

partidas_response = requests.get('https://api.cartola.globo.com/partidas')
partidas_json_data = partidas_response.json()
partidas_df = pd.json_normalize(partidas_json_data["partidas"],max_level=1)
print(partidas_df.columns.tolist())
confrontos_df = partidas_df.merge(clubes_df.add_prefix('casa_'), left_on='clube_casa_id', right_on='casa_clube_id')
confrontos_df = confrontos_df.merge(clubes_df.add_prefix('visitante_'), left_on='clube_visitante_id', right_on='visitante_clube_id')
confrontos_df['confronto_slug'] = confrontos_df['casa_clube_abreviacao'] + ' X ' + confrontos_df['visitante_clube_abreviacao']
print(confrontos_df.columns.tolist())

['atleta_minimo_para_valorizar', 'atleta_jogos_num', 'atleta_atleta_id', 'atleta_rodada_id', 'atleta_clube_id', 'atleta_posicao_id', 'atleta_status_id', 'atleta_pontos_num', 'atleta_media_num', 'atleta_variacao_num', 'atleta_preco_num', 'atleta_entrou_em_campo', 'atleta_slug', 'atleta_apelido', 'atleta_apelido_abreviado', 'atleta_nome', 'atleta_foto', 'atleta_scout.DE', 'atleta_scout.FS', 'atleta_scout.GS', 'atleta_scout.SG', 'atleta_gato_mestre.scouts.media', 'atleta_gato_mestre.scouts.mandante', 'atleta_gato_mestre.scouts.visitante', 'atleta_gato_mestre.media_pontos_mandante', 'atleta_gato_mestre.media_pontos_visitante', 'atleta_gato_mestre.media_minutos_jogados', 'atleta_gato_mestre.minutos_jogados', 'atleta_scout.CA', 'atleta_scout.A', 'atleta_scout.DS', 'atleta_scout.FC', 'atleta_scout.FD', 'atleta_scout.FF', 'atleta_scout.FT', 'atleta_scout.I', 'atleta_scout.CV', 'atleta_scout.G', 'atleta_scout.PS', 'atleta_scout.V', 'atleta_scout.PP', 'atleta_scout.PC', 'atleta_scout.DP', 'atlet

In [13]:
df = atletas_df.merge(posicoes_df, left_on='posicao_id',right_on='posicao_id')
df = df.merge(clubes_df, left_on='clube_id', right_on='clube_id')
df = df.merge(status_df, left_on='status_id', right_on='status_id')
df = df.merge(confrontos_df, how='left', left_on=['clube_id'], right_on=['clube_casa_id'])
df = df.merge(confrontos_df, how='left', left_on=['clube_id'], right_on=['clube_visitante_id'])
df['confronto_slug'] = df['confronto_slug_y'].fillna('') + '' + df['confronto_slug_x'].fillna('')
print(df[['confronto_slug']])

    confronto_slug
0        CRU X VAS
1        PAL X CAM
2        CRU X VAS
3        PAL X CAM
4        FLA X CAP
..             ...
775      CRU X VAS
776      PAL X CAM
777      SAO X COR
778      SAO X COR
779      CRU X VAS

[780 rows x 1 columns]


In [14]:
df_filtrado = df[df['status_nome'] == 'Provável']
print(df_filtrado[colunas])

      atleta_apelido clube_abreviacao posicao_abreviacao status_nome  \
0         Léo Jardim              VAS                gol    Provável   
1            Everson              CAM                gol    Provável   
2              Payet              VAS                mei    Provável   
3           Paulinho              CAM                ata    Provável   
4          Mastriani              CAP                ata    Provável   
..               ...              ...                ...         ...   
767         Carrillo              JUV                ata    Provável   
770           Almada              BOT                mei    Provável   
771  Felipe Anderson              PAL                mei    Provável   
776          Bernard              CAM                mei    Provável   
778    André Ramalho              COR                zag    Provável   

     atleta_preco_num  
0                6.07  
1                8.08  
2                5.62  
3               10.12  
4              

In [15]:
print(df_filtrado.columns.tolist())

['atleta_minimo_para_valorizar', 'atleta_jogos_num', 'atleta_id', 'atleta_rodada_id', 'clube_id', 'posicao_id', 'status_id', 'atleta_pontos_num', 'atleta_media_num', 'atleta_variacao_num', 'atleta_preco_num', 'atleta_entrou_em_campo', 'atleta_slug', 'atleta_apelido', 'atleta_apelido_abreviado', 'atleta_nome', 'atleta_foto', 'atleta_scout.DE', 'atleta_scout.FS', 'atleta_scout.GS', 'atleta_scout.SG', 'atleta_gato_mestre.scouts.media', 'atleta_gato_mestre.scouts.mandante', 'atleta_gato_mestre.scouts.visitante', 'atleta_gato_mestre.media_pontos_mandante', 'atleta_gato_mestre.media_pontos_visitante', 'atleta_gato_mestre.media_minutos_jogados', 'atleta_gato_mestre.minutos_jogados', 'atleta_scout.CA', 'atleta_scout.A', 'atleta_scout.DS', 'atleta_scout.FC', 'atleta_scout.FD', 'atleta_scout.FF', 'atleta_scout.FT', 'atleta_scout.I', 'atleta_scout.CV', 'atleta_scout.G', 'atleta_scout.PS', 'atleta_scout.V', 'atleta_scout.PP', 'atleta_scout.PC', 'atleta_scout.DP', 'atleta_scout.GC', 'posicao_nome',

In [16]:
# prompt: gere uma analises das medias de cada posicao_id onde a coluna atleta_media_num seja diferente de 0 e mostre a media de cada posicao

# Create a new dataframe with only the relevant columns
df_analise = df[['posicao_id', 'atleta_media_num','atleta_entrou_em_campo']]

# Filter out rows where atleta_media_num is 0
df_analise = df_analise[df_analise['atleta_entrou_em_campo'] == True]

# Group the dataframe by posicao_id and calculate the mean of atleta_media_num
medias_posicao = df_analise.groupby('posicao_id')['atleta_media_num'].mean()

# Print the results
print(medias_posicao)

posicao_id
1    3.899000
2    3.709362
3    2.868043
4    2.832222
5    3.315730
6    4.953889
Name: atleta_media_num, dtype: float64


In [17]:
# prompt: gere uma media dos clubes usando a coluna clube_abreviacao a media devera ser o atleta_media_num diferente de 0 ordene pela media descendente

# Create a new dataframe with only the relevant columns
df_analise = df[['clube_abreviacao', 'atleta_media_num','atleta_entrou_em_campo']]

# Filter out rows where atleta_media_num is 0
df_analise = df_analise[df_analise['atleta_entrou_em_campo'] == True]

# Group the dataframe by clube_abreviacao and calculate the mean of atleta_media_num
medias_clube = df_analise.groupby('clube_abreviacao')['atleta_media_num'].mean()

# Sort the results in descending order
medias_clube = medias_clube.sort_values(ascending=False)

# Print the results
print(medias_clube)

clube_abreviacao
BOT    3.990000
PAL    3.967647
GRE    3.794118
CRU    3.653750
FOR    3.563529
INT    3.532941
SAO    3.441176
JUV    3.432941
COR    3.334118
CAM    3.266471
BAH    3.238235
VAS    3.182353
VIT    3.071765
CRI    3.025882
FLU    2.919444
CAP    2.913125
BGT    2.884118
FLA    2.884000
ACG    2.632353
CUI    2.433750
Name: atleta_media_num, dtype: float64


# Estudos
posicao_id | posicao_nome | posicao_abreviacao
- | -| -
1 | Goleiro | gol
2 | Lateral | lat
3 | Zagueiro | zag
4 | Meia | mei
5 | Atacante | ata
6 | Técnico | tec

## Atacante

In [18]:
import numpy as np
colunas_relatorio = ['atleta_apelido','clube_abreviacao','confronto_slug','atleta_media_num','atleta_apelido','atleta_pontos_num', 'posicao_abreviacao', 'atleta_scout.G']
posicao_id = 5 #atacante
df_analise = df_filtrado[df_filtrado['posicao_id'] == posicao_id]
medias_posicao = df_analise['atleta_media_num'].mean()
df_analise['dif'] = df_analise['atleta_media_num'] - medias_posicao
df_analise = df_analise[df_analise['atleta_pontos_num'] != 0]
df_analise = df_analise[df_analise['atleta_media_num'] > medias_posicao]
df_relatorio = df_analise[colunas_relatorio]
df_relatorio = df_relatorio.sort_values(by=['atleta_pontos_num'])
print(df_relatorio)

    atleta_apelido clube_abreviacao confronto_slug  atleta_media_num  \
609  William Gomes              SAO      SAO X COR              5.13   
94   Luiz Fernando              ACG      ACG X FLU              4.66   
9          Vegetti              VAS      CRU X VAS              6.04   
452          Serna              FLU      ACG X FLU              5.15   
757     Igor Jesus              BOT      BOT X GRE              6.02   
53         Soteldo              GRE      BOT X GRE              4.62   
165  Eduardo Sasha              BGT      JUV X BGT              4.42   
624        Bolasie              CRI      BAH X CRI              5.75   
367          Borré              INT      INT X VIT              6.79   
193   Yuri Alberto              COR      SAO X COR              4.93   
374         Wesley              INT      INT X VIT              6.63   
478    Braithwaite              GRE      BOT X GRE              9.50   
5             Hulk              CAM      PAL X CAM              

## Meias

In [19]:
# prompt: quero obter do dataframe_filtrado todos os jogadores que possuem atleta_media_num acima da media da atleta _posicao_id onde o atleta_variacao seja < 0 ordene pela atleta_variacao_num

posicao_id = 4 #meias
df_analise = df_filtrado[df_filtrado['posicao_id'] == posicao_id]
medias_posicao = df_analise['atleta_media_num'].mean()
df_analise['dif'] = df_analise['atleta_media_num'] - medias_posicao
df_analise = df_analise[df_analise['atleta_pontos_num'] != 0]
df_analise = df_analise[df_analise['atleta_media_num'] > medias_posicao]
df_relatorio = df_analise[colunas_relatorio]
df_relatorio = df_relatorio.sort_values(by=['atleta_pontos_num'])
print(df_relatorio)

        atleta_apelido clube_abreviacao confronto_slug  atleta_media_num  \
238    Matheus Pereira              CRU      CRU X VAS              5.66   
140         Jean Lucas              BAH      BAH X CRI              4.66   
135           Thaciano              BAH      BAH X CRI              4.04   
6       Gustavo Scarpa              CAM      PAL X CAM              5.92   
382             Jadson              JUV      JUV X BGT              4.87   
654        Matheusinho              CRI      BAH X CRI              4.61   
550            Ronaldo              JUV      JUV X BGT              7.80   
50          Villasanti              GRE      BOT X GRE              4.14   
289              Ganso              FLU      ACG X FLU              3.45   
425          Jhon Jhon              BGT      JUV X BGT              4.13   
766           Monsalve              GRE      BOT X GRE              6.99   
56     Everton Ribeiro              BAH      BAH X CRI              4.27   
466         

## Zagueiros

In [20]:
posicao_id = 3 #zagueiros
df_analise = df_filtrado[df_filtrado['posicao_id'] == posicao_id]
medias_posicao = df_analise['atleta_media_num'].mean()
df_analise['dif'] = df_analise['atleta_media_num'] - medias_posicao
df_analise = df_analise[df_analise['atleta_pontos_num'] != 0]
df_analise = df_analise[df_analise['atleta_media_num'] > medias_posicao]
df_relatorio = df_analise[colunas_relatorio]
df_relatorio = df_relatorio.sort_values(by=['atleta_pontos_num'])
print(df_relatorio)

        atleta_apelido clube_abreviacao confronto_slug  atleta_media_num  \
144     Gabriel Xavier              BAH      BAH X CRI              3.56   
101    Adriano Martins              ACG      ACG X FLU              3.00   
370              Vitão              INT      INT X VIT              3.02   
92              Bastos              BOT      BOT X GRE              3.93   
196               Cacá              COR      SAO X COR              3.43   
34              Murilo              PAL      PAL X CAM              4.04   
245           Villalba              CRU      CRU X VAS              3.49   
543    Wagner Leonardo              VIT      INT X VIT              3.36   
157  Alexander Barboza              BOT      BOT X GRE              3.50   
763      Junior Alonso              CAM      PAL X CAM              4.55   
74        Kaíque Rocha              CAP      FLA X CAP              3.25   
324             Brítez              FOR      FOR X CUI              3.37   
31       Gus

## Lateral

posicao_id = 2 #laterais
df_analise = df_filtrado[df_filtrado['posicao_id'] == posicao_id]
medias_posicao = df_analise['atleta_media_num'].mean()
df_analise['dif'] = df_analise['atleta_media_num'] - medias_posicao
df_analise = df_analise[df_analise['atleta_pontos_num'] != 0]
df_analise = df_analise[df_analise['atleta_media_num'] > medias_posicao]
df_relatorio = df_analise[colunas_relatorio]
df_relatorio = df_relatorio.sort_values(by=['atleta_pontos_num'])
print(df_relatorio)

## Goleiro

In [21]:
posicao_id = 1 #goleiros
df_analise = df_filtrado[df_filtrado['posicao_id'] == posicao_id]
medias_posicao = df_analise['atleta_media_num'].mean()
df_analise['dif'] = df_analise['atleta_media_num'] - medias_posicao
df_analise = df_analise[df_analise['atleta_pontos_num'] != 0]
df_analise = df_analise[df_analise['atleta_media_num'] > medias_posicao]
df_relatorio = df_analise[colunas_relatorio]
df_relatorio = df_relatorio.sort_values(by=['atleta_pontos_num'])
print(df_relatorio)

    atleta_apelido clube_abreviacao confronto_slug  atleta_media_num  \
315   João Ricardo              FOR      FOR X CUI              4.98   
334      Marchesín              GRE      BOT X GRE              4.10   
133  Marcos Felipe              BAH      BAH X CRI              4.16   
284          Fábio              FLU      ACG X FLU              4.28   
405       Weverton              PAL      PAL X CAM              5.63   
166        Cleiton              BGT      JUV X BGT              3.71   
187         Cássio              CRU      CRU X VAS              5.12   
733     Hugo Souza              COR      SAO X COR              4.34   
472           John              BOT      BOT X GRE              4.47   

    atleta_apelido  atleta_pontos_num posicao_abreviacao  atleta_scout.G  
315   João Ricardo                2.0                gol             NaN  
334      Marchesín                4.0                gol             NaN  
133  Marcos Felipe                5.0                g

## Técnicos

In [22]:
posicao_id = 6 #tecnicos
df_analise = df_filtrado[df_filtrado['posicao_id'] == posicao_id]
medias_posicao = df_analise['atleta_media_num'].mean()
df_analise['dif'] = df_analise['atleta_media_num'] - medias_posicao
df_analise = df_analise[df_analise['atleta_pontos_num'] != 0]
df_analise = df_analise[df_analise['atleta_media_num'] > medias_posicao]
df_relatorio = df_analise[colunas_relatorio]
df_relatorio = df_relatorio.sort_values(by=['atleta_pontos_num'])
print(df_relatorio)

     atleta_apelido clube_abreviacao confronto_slug  atleta_media_num  \
691    Rafael Paiva              VAS      CRU X VAS              5.51   
681   Luis Zubeldía              SAO      SAO X COR              5.10   
727    Mano Menezes              FLU      ACG X FLU              5.49   
270            Tite              FLA      FLA X CAP              5.39   
150    Rogério Ceni              BAH      BAH X CRI              5.19   
416   Abel Ferreira              PAL      PAL X CAM              6.47   
332    Juan Vojvoda              FOR      FOR X CUI              5.43   
383   Roger Machado              INT      INT X VIT              5.14   
647     Artur Jorge              BOT      BOT X GRE              5.87   
626  Gabriel Milito              CAM      PAL X CAM              4.89   
7        Ramón Díaz              COR      SAO X COR              5.22   

     atleta_apelido  atleta_pontos_num posicao_abreviacao  atleta_scout.G  
691    Rafael Paiva               2.00         